In [1]:
import os
import networkx as nx
import json
from datetime import datetime
import dateutil.parser as parser
import pandas as pd
import sys
import multiprocessing as mp
import numpy as np
import pickle

In [2]:
TRACE_DATA_DIR = '/mnt/raid0_ssd_8tb/rimon/gemini1_datasets/trace/'

In [3]:
all_trace_files = [os.path.join(TRACE_DATA_DIR, f) for f in os.listdir(TRACE_DATA_DIR)]

In [5]:
temp1 = all_trace_files[0]
temp2 = all_trace_files[1]

In [7]:
df1 = pd.read_json(temp1, lines=True)
df2 = pd.read_json(temp2, lines=True)

In [55]:
df1

,datum,CDMVersion,source
0,{'com.bbn.tc.schema.avro.cdm18.Event': {'uuid'...,18,SOURCE_LINUX_SYSCALL_TRACE
1,{'com.bbn.tc.schema.avro.cdm18.SrcSinkObject':...,18,SOURCE_LINUX_SYSCALL_TRACE
2,{'com.bbn.tc.schema.avro.cdm18.Event': {'uuid'...,18,SOURCE_LINUX_SYSCALL_TRACE
3,{'com.bbn.tc.schema.avro.cdm18.SrcSinkObject':...,18,SOURCE_LINUX_SYSCALL_TRACE
4,{'com.bbn.tc.schema.avro.cdm18.Event': {'uuid'...,18,SOURCE_LINUX_SYSCALL_TRACE
...,...,...,...
4999995,{'com.bbn.tc.schema.avro.cdm18.Subject': {'uui...,18,SOURCE_LINUX_BEEP_TRACE
4999996,{'com.bbn.tc.schema.avro.cdm18.Event': {'uuid'...,18,SOURCE_LINUX_BEEP_TRACE
4999997,{'com.bbn.tc.schema.avro.cdm18.Subject': {'uui...,18,SOURCE_LINUX_BEEP_TRACE
4999998,{'com.bbn.tc.schema.avro.cdm18.Event': {'uuid'...,18,SOURCE_LINUX_BEEP_TRACE


In [58]:
df2['datum'].iloc[0]

{'com.bbn.tc.schema.avro.cdm18.Event': {'uuid': '0E695069-A079-882E-9F89-AB321FDBF1C9',
  'sequence': {'long': 1937130142},
  'type': 'EVENT_MPROTECT',
  'threadId': {'int': 22247},
  'hostId': '3120B2A9-057E-E4EB-4BB9-154983D2C063',
  'subject': {'com.bbn.tc.schema.avro.cdm18.UUID': '204EB826-1787-B9D1-CBD9-2D9015AA598E'},
  'predicateObject': {'com.bbn.tc.schema.avro.cdm18.UUID': '00EAB82E-BA0D-52DE-3394-486F11558ED8'},
  'predicateObjectPath': None,
  'predicateObject2': None,
  'predicateObject2Path': None,
  'timestampNanos': 1523469878202000000,
  'name': None,
  'parameters': None,
  'location': None,
  'size': None,
  'programPoint': None,
  'properties': {'map': {'protection': '5'}}}}

In [15]:
unique_datum1 = set([])
df1.apply(lambda x: unique_datum1.add(next(iter(x['datum']))), axis=1)

0          None
1          None
2          None
3          None
4          None
           ... 
4999995    None
4999996    None
4999997    None
4999998    None
4999999    None
Length: 5000000, dtype: object

In [17]:
unique_datum2 = set([])
df2.apply(lambda x: unique_datum2.add(next(iter(x['datum']))), axis=1)

0          None
1          None
2          None
3          None
4          None
           ... 
4999995    None
4999996    None
4999997    None
4999998    None
4999999    None
Length: 5000000, dtype: object

In [20]:
print(unique_datum1 == unique_datum2)
unique_datum1

True


{'com.bbn.tc.schema.avro.cdm18.Event',
 'com.bbn.tc.schema.avro.cdm18.FileObject',
 'com.bbn.tc.schema.avro.cdm18.MemoryObject',
 'com.bbn.tc.schema.avro.cdm18.NetFlowObject',
 'com.bbn.tc.schema.avro.cdm18.SrcSinkObject',
 'com.bbn.tc.schema.avro.cdm18.Subject',
 'com.bbn.tc.schema.avro.cdm18.UnitDependency',
 'com.bbn.tc.schema.avro.cdm18.UnnamedPipeObject'}

In [ ]:
df1['datum'].apply(lambda x: list(x.keys())).explode().value_counts()

com.bbn.tc.schema.avro.cdm18.Event                3344976
com.bbn.tc.schema.avro.cdm18.SrcSinkObject         675012
com.bbn.tc.schema.avro.cdm18.Subject               453701
com.bbn.tc.schema.avro.cdm18.UnitDependency        394325
com.bbn.tc.schema.avro.cdm18.MemoryObject           88522
com.bbn.tc.schema.avro.cdm18.NetFlowObject          28082
com.bbn.tc.schema.avro.cdm18.FileObject             14118
com.bbn.tc.schema.avro.cdm18.UnnamedPipeObject       1264
Name: datum, dtype: int64

In [52]:
df2['datum'].apply(lambda x: list(x.keys())).explode().value_counts()

com.bbn.tc.schema.avro.cdm18.Event                3262915
com.bbn.tc.schema.avro.cdm18.SrcSinkObject         666849
com.bbn.tc.schema.avro.cdm18.Subject               500649
com.bbn.tc.schema.avro.cdm18.UnitDependency        467589
com.bbn.tc.schema.avro.cdm18.MemoryObject           56515
com.bbn.tc.schema.avro.cdm18.NetFlowObject          32894
com.bbn.tc.schema.avro.cdm18.FileObject             11845
com.bbn.tc.schema.avro.cdm18.UnnamedPipeObject        744
Name: datum, dtype: int64

In [54]:
print(3344976 / 5000000)
print(3262915 / 5000000)
# around 66% are event objects - good

0.6689952
0.652583


In [59]:
# do some timestamp analysis 
normalized_df1 = pd.json_normalize(df1['datum'])

In [60]:
normalized_df1

,com.bbn.tc.schema.avro.cdm18.Event.uuid,com.bbn.tc.schema.avro.cdm18.Event.sequence.long,com.bbn.tc.schema.avro.cdm18.Event.type,com.bbn.tc.schema.avro.cdm18.Event.threadId.int,com.bbn.tc.schema.avro.cdm18.Event.hostId,com.bbn.tc.schema.avro.cdm18.Event.subject.com.bbn.tc.schema.avro.cdm18.UUID,com.bbn.tc.schema.avro.cdm18.Event.predicateObject.com.bbn.tc.schema.avro.cdm18.UUID,com.bbn.tc.schema.avro.cdm18.Event.predicateObjectPath,com.bbn.tc.schema.avro.cdm18.Event.predicateObject2,com.bbn.tc.schema.avro.cdm18.Event.predicateObject2Path,...,com.bbn.tc.schema.avro.cdm18.UnnamedPipeObject.baseObject.epoch.int,com.bbn.tc.schema.avro.cdm18.UnnamedPipeObject.baseObject.properties.map.pid,com.bbn.tc.schema.avro.cdm18.UnnamedPipeObject.sourceFileDescriptor.int,com.bbn.tc.schema.avro.cdm18.UnnamedPipeObject.sinkFileDescriptor.int,com.bbn.tc.schema.avro.cdm18.UnnamedPipeObject.sourceUUID,com.bbn.tc.schema.avro.cdm18.UnnamedPipeObject.sinkUUID,com.bbn.tc.schema.avro.cdm18.Event.properties.map.operation,com.bbn.tc.schema.avro.cdm18.Subject.parentSubject,com.bbn.tc.schema.avro.cdm18.Subject.properties.map.seen time,com.bbn.tc.schema.avro.cdm18.Event.location.long
0,B7CF3A75-822B-CA7B-9FAF-074E5EF1FE90,1.908210e+09,EVENT_SENDMSG,21790.0,3120B2A9-057E-E4EB-4BB9-154983D2C063,6881BEAE-AEDC-B80D-F325-67C69EA9D711,17B28CB0-EAC8-10B3-5BE1-B672D611BD9B,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7FEDB19D-70FE-A735-10AC-C1D2AF37312C,1.908210e+09,EVENT_RECVMSG,21792.0,3120B2A9-057E-E4EB-4BB9-154983D2C063,5BA646C2-52E0-FD93-4763-09358A07202A,0E06BFEC-893C-1E56-0122-2B3FF4E00701,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,44F8544B-E70A-F803-196F-50963746BE35,1.908210e+09,EVENT_SENDMSG,21792.0,3120B2A9-057E-E4EB-4BB9-154983D2C063,5BA646C2-52E0-FD93-4763-09358A07202A,5DD8F7FB-96F3-E61D-24D7-E4DD70508B2D,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4999996,9A9D80D7-FCBE-8D1D-3079-00F2755167AC,1.926364e+09,EVENT_UNIT,22208.0,3120B2A9-057E-E4EB-4BB9-154983D2C063,8E0F9235-C338-73C4-6B46-411935CC7D18,39D9CDA5-BFCB-0BD4-58E0-17A0CDB8F62A,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4999997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4999998,26D5D084-06F9-EA33-10C4-25D64728075A,1.926364e+09,EVENT_UNIT,22193.0,3120B2A9-057E-E4EB-4BB9-154983D2C063,53D666B1-70C2-F82F-FE45-E9FF6F00EE6A,091BF32A-DC5D-8DFA-3D9C-9F936FDB205D,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
normalized_df1.min(axis=0)

/tmp/ipykernel_1734869/1322253572.py:1: FutureWarning: The default value of numeric_only in DataFrame.min is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  normalized_df1.min(axis=0)


com.bbn.tc.schema.avro.cdm18.Event.sequence.long                           1.908210e+09
com.bbn.tc.schema.avro.cdm18.Event.threadId.int                            1.051000e+03
com.bbn.tc.schema.avro.cdm18.Event.predicateObjectPath                              NaN
com.bbn.tc.schema.avro.cdm18.Event.predicateObject2                                 NaN
com.bbn.tc.schema.avro.cdm18.Event.predicateObject2Path                             NaN
com.bbn.tc.schema.avro.cdm18.Event.timestampNanos                          1.523462e+18
com.bbn.tc.schema.avro.cdm18.Event.name                                             NaN
com.bbn.tc.schema.avro.cdm18.Event.parameters                                       NaN
com.bbn.tc.schema.avro.cdm18.Event.location                                         NaN
com.bbn.tc.schema.avro.cdm18.Event.size.long                               0.000000e+00
com.bbn.tc.schema.avro.cdm18.Event.programPoint                                     NaN
com.bbn.tc.schema.avro.cdm18.Src

In [63]:
normalized_df1.max(axis=0)

/tmp/ipykernel_1734869/2872451825.py:1: FutureWarning: The default value of numeric_only in DataFrame.max is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  normalized_df1.max(axis=0)


com.bbn.tc.schema.avro.cdm18.Event.sequence.long                           1.926364e+09
com.bbn.tc.schema.avro.cdm18.Event.threadId.int                            2.991300e+04
com.bbn.tc.schema.avro.cdm18.Event.predicateObjectPath                              NaN
com.bbn.tc.schema.avro.cdm18.Event.predicateObject2                                 NaN
com.bbn.tc.schema.avro.cdm18.Event.predicateObject2Path                             NaN
com.bbn.tc.schema.avro.cdm18.Event.timestampNanos                          1.523465e+18
com.bbn.tc.schema.avro.cdm18.Event.name                                             NaN
com.bbn.tc.schema.avro.cdm18.Event.parameters                                       NaN
com.bbn.tc.schema.avro.cdm18.Event.location                                         NaN
com.bbn.tc.schema.avro.cdm18.Event.size.long                               3.295642e+06
com.bbn.tc.schema.avro.cdm18.Event.programPoint                                     NaN
com.bbn.tc.schema.avro.cdm18.Src

In [6]:
# min timestamp for events from df1: Wed Apr 11 2018 11:53:20 GMT-0400 (Eastern Daylight Time)
# max timestamp for events from df1: Wed Apr 11 2018 12:43:20 GMT-0400 (Eastern Daylight Time)

In [1]:
TRACE_DATA_DIR = '/mnt/raid0_ssd_8tb/rimon/gemini1_datasets/trace/'

In [16]:
timestamps_df = pd.read_csv('../data/timestamps.txt', sep='\t')

In [21]:
timestamps_df['min_datetime'] = timestamps_df.apply(lambda x: datetime.fromtimestamp(x['min_time'] // 1000000000), axis=1)
timestamps_df['max_datetime'] = timestamps_df.apply(lambda x: datetime.fromtimestamp(x['max_time'] // 1000000000), axis=1)

In [25]:
mask_10th = (timestamps_df['min_time'] > 1523332800 * 1000000000) & (timestamps_df['max_time'] < 1523419199 * 1000000000)

In [28]:
# TRACE Drakon Firefox backdoor with drakon APT in memory on April 10th started @ 10 AM
# 3.2 20180410 1000 TRACE – Firefox Backdoor w/ Drakon In-Memory

timestamps_df[mask_10th].sort_values('min_time')

,file,min_time,max_time,min_datetime,max_datetime
12,/mnt/raid0_ssd_8tb/rimon/gemini1_datasets/trac...,1523336194113000000,1523343513168000000,2018-04-10 00:56:34,2018-04-10 02:58:33
3,/mnt/raid0_ssd_8tb/rimon/gemini1_datasets/trac...,1523350646120000000,1523357764428000000,2018-04-10 04:57:26,2018-04-10 06:56:04
28,/mnt/raid0_ssd_8tb/rimon/gemini1_datasets/trac...,1523357764428000000,1523362902620000000,2018-04-10 06:56:04,2018-04-10 08:21:42
13,/mnt/raid0_ssd_8tb/rimon/gemini1_datasets/trac...,1523358607461000000,1523407023043000000,2018-04-10 07:10:07,2018-04-10 20:37:03
48,/mnt/raid0_ssd_8tb/rimon/gemini1_datasets/trac...,1523362902620000000,1523368211327000000,2018-04-10 08:21:42,2018-04-10 09:50:11
54,/mnt/raid0_ssd_8tb/rimon/gemini1_datasets/trac...,1523365062110000000,1523414635961000000,2018-04-10 08:57:42,2018-04-10 22:43:55
19,/mnt/raid0_ssd_8tb/rimon/gemini1_datasets/trac...,1523368211327000000,1523368223807000000,2018-04-10 09:50:11,2018-04-10 09:50:23
68,/mnt/raid0_ssd_8tb/rimon/gemini1_datasets/trac...,1523370537991000000,1523377398805000000,2018-04-10 10:28:57,2018-04-10 12:23:18
4,/mnt/raid0_ssd_8tb/rimon/gemini1_datasets/trac...,1523377398805000000,1523384612090000000,2018-04-10 12:23:18,2018-04-10 14:23:32
24,/mnt/raid0_ssd_8tb/rimon/gemini1_datasets/trac...,1523384612090000000,1523391700075000000,2018-04-10 14:23:32,2018-04-10 16:21:40


In [33]:
temp = timestamps_df[mask_10th].sort_values('min_time')['file']
temp.values.flatten().tolist()

['/mnt/raid0_ssd_8tb/rimon/gemini1_datasets/trace/ta1-trace-e3-official.json.118',
 '/mnt/raid0_ssd_8tb/rimon/gemini1_datasets/trace/ta1-trace-e3-official.json.120',
 '/mnt/raid0_ssd_8tb/rimon/gemini1_datasets/trace/ta1-trace-e3-official.json.121',
 '/mnt/raid0_ssd_8tb/rimon/gemini1_datasets/trace/ta1-trace-e3-official.json.129',
 '/mnt/raid0_ssd_8tb/rimon/gemini1_datasets/trace/ta1-trace-e3-official.json.122',
 '/mnt/raid0_ssd_8tb/rimon/gemini1_datasets/trace/ta1-trace-e3-official.json.130',
 '/mnt/raid0_ssd_8tb/rimon/gemini1_datasets/trace/ta1-trace-e3-official.json.123',
 '/mnt/raid0_ssd_8tb/rimon/gemini1_datasets/trace/ta1-trace-e3-official.json.125',
 '/mnt/raid0_ssd_8tb/rimon/gemini1_datasets/trace/ta1-trace-e3-official.json.126',
 '/mnt/raid0_ssd_8tb/rimon/gemini1_datasets/trace/ta1-trace-e3-official.json.127',
 '/mnt/raid0_ssd_8tb/rimon/gemini1_datasets/trace/ta1-trace-e3-official.json.128']

# Spark Processing

In [6]:
import networkx as nx
import json
from datetime import datetime
import dateutil.parser as parser
import pandas as pd
import sys
import multiprocessing as mp
import pickle

import os
import pyspark
import numpy as np
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
from datetime import datetime

In [7]:
spark = SparkSession.builder.master("local[20]").appName("tc_explore.com").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [8]:
df = spark.read.json(all_trace_files)

23/01/30 14:25:23 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [42]:
df.printSchema()

root
 |-- CDMVersion: string (nullable = true)
 |-- datum: struct (nullable = true)
 |    |-- com.bbn.tc.schema.avro.cdm18.EndMarker: struct (nullable = true)
 |    |    |-- recordCounts: struct (nullable = true)
 |    |    |    |-- EVENT_ACCEPT: string (nullable = true)
 |    |    |    |-- EVENT_CHANGE_PRINCIPAL: string (nullable = true)
 |    |    |    |-- EVENT_CLONE: string (nullable = true)
 |    |    |    |-- EVENT_CLOSE: string (nullable = true)
 |    |    |    |-- EVENT_CONNECT: string (nullable = true)
 |    |    |    |-- EVENT_CREATE_OBJECT: string (nullable = true)
 |    |    |    |-- EVENT_EXECUTE: string (nullable = true)
 |    |    |    |-- EVENT_EXIT: string (nullable = true)
 |    |    |    |-- EVENT_FORK: string (nullable = true)
 |    |    |    |-- EVENT_LINK: string (nullable = true)
 |    |    |    |-- EVENT_LOADLIBRARY: string (nullable = true)
 |    |    |    |-- EVENT_MMAP: string (nullable = true)
 |    |    |    |-- EVENT_MODIFY_FILE_ATTRIBUTES: string (nullabl

In [41]:
times = df.select("datum.*").select("`com.bbn.tc.schema.avro.cdm18.Subject`").withColumnRenamed("com.bbn.tc.schema.avro.cdm18.Subject", "subject").select("subject.*")
times = times.filter('startTimestampNanos IS NOT NULL')
times.count() # = 39318719

39318719

In [44]:
times.agg({'startTimestampNanos': 'max'}).collect()[0]

Row(max(startTimestampNanos)=1523653792143000000)

In [45]:
times.agg({'startTimestampNanos': 'min'}).collect()[0]

Row(min(startTimestampNanos)=0)

In [46]:
new_times = times.filter('startTimestampNanos > 0')
new_times.agg({'startTimestampNanos': 'min'}).collect()[0]

Row(min(startTimestampNanos)=1522703821498000000)

In [47]:
'''
4/10 @ 10:00 - Firefox backdoor with drakon APT in memory
4-12 @ 14:00 - Nginx backdoor with drakon APT in memory
4-13 @ 12:00 - Browser extension with drakon APT on disk
'''

'\n4/10 @ 10:00 - Firefox backdoor with drakon APT in memory\n4-12 @ 14:00 - Nginx backdoor with drakon APT in memory\n4-13 @ 12:00 - Browser extension with drakon APT on disk\n'

In [56]:
# select all times on the 10th - lets examine what there is here
# min time on the 10th (GMT): 1523318400
# max time on the 10th (GMT): 1523404800
start_time = 1523318400 * 1000000000
end_time = 1523404800 * 1000000000
subjects_10th = new_times.filter("startTimestampNanos > {} AND startTimestampNanos < {}".format(start_time, end_time))
subjects_10th

Exception in thread "serve-GetRows" java.net.SocketTimeoutException: Accept timed out
	at java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:409)
	at java.net.ServerSocket.implAccept(ServerSocket.java:560)
	at java.net.ServerSocket.accept(ServerSocket.java:528)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:64)


cid,cmdLine,count,exportedLibraries,hostId,importedLibraries,iteration,localPrincipal,parentSubject,privilegeLevel,properties,startTimestampNanos,type,unitId,uuid
24502,{/usr/local/firef...,null,null,3120B2A9-057E-E4E...,null,null,29895546-B124-1BE...,null,null,"{{/home/admin, We...",1523370613699000000,SUBJECT_PROCESS,{0},D4EFE0BC-99A8-CEE...
24767,{/usr/local/firef...,null,null,3120B2A9-057E-E4E...,null,null,29895546-B124-1BE...,null,null,"{{/home/admin, We...",1523370698960000000,SUBJECT_PROCESS,{0},45DD25D8-AD01-E41...
24797,{firefox},null,null,3120B2A9-057E-E4E...,null,null,42C80B68-B550-026...,null,null,"{{/home/admin, fi...",1523327092006000000,SUBJECT_PROCESS,{0},44BE575D-28E4-E92...
25301,{/usr/local/firef...,null,null,3120B2A9-057E-E4E...,null,null,42C80B68-B550-026...,null,null,"{{/home/admin, We...",1523371035432000000,SUBJECT_PROCESS,{0},268B02E3-B19A-FD0...
25302,{/usr/local/firef...,null,null,3120B2A9-057E-E4E...,null,null,42C80B68-B550-026...,null,null,"{{/home/admin, We...",1523371035432000000,SUBJECT_PROCESS,{0},CEA0C5FB-769F-FCA...
25309,{/usr/local/firef...,null,null,3120B2A9-057E-E4E...,null,null,42C80B68-B550-026...,null,null,"{{/home/admin, We...",1523371035476000000,SUBJECT_PROCESS,{0},85444F52-A982-564...
25428,{/usr/local/firef...,null,null,3120B2A9-057E-E4E...,null,null,29895546-B124-1BE...,null,null,"{{/home/admin, We...",1523371064080000000,SUBJECT_PROCESS,{0},A3CF0A17-2723-9C1...
25640,{/usr/local/firef...,null,null,3120B2A9-057E-E4E...,null,null,42C80B68-B550-026...,null,null,"{{/home/admin, We...",1523371190861000000,SUBJECT_PROCESS,{0},F20FEC3C-8DC9-CF8...
32502,{fuzzyflakes -root},null,null,3120B2A9-057E-E4E...,null,null,29895546-B124-1BE...,{94CF1DAE-4884-CA...,null,"{{/home/admin, fu...",1523341453295000000,SUBJECT_PROCESS,{0},0D305384-A5FC-E80...
14478,{/usr/local/firef...,null,null,3120B2A9-057E-E4E...,null,null,42C80B68-B550-026...,null,null,"{{/home/admin, We...",1523362771639000000,SUBJECT_PROCESS,{0},470F9BDD-A889-BB4...


In [59]:
subjects_10th.count()

1314936

In [58]:
events_10th = df.select("datum.*").select("`com.bbn.tc.schema.avro.cdm18.Event`").withColumnRenamed("com.bbn.tc.schema.avro.cdm18.Event", "event").select("event.*")
events_10th

hostId,location,name,parameters,predicateObject,predicateObject2,predicateObject2Path,predicateObjectPath,programPoint,properties,sequence,size,subject,threadId,timestampNanos,type,uuid
3120B2A9-057E-E4E...,null,null,null,{17B28CB0-EAC8-10...,null,null,null,null,"{{null, null, nul...",{1908210262},{102},{6881BEAE-AEDC-B8...,{21790},1523462095074000000,EVENT_SENDMSG,B7CF3A75-822B-CA7...
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
3120B2A9-057E-E4E...,null,null,null,{0E06BFEC-893C-1E...,null,null,null,null,"{{null, null, nul...",{1908210263},{102},{5BA646C2-52E0-FD...,{21792},1523462095074000000,EVENT_RECVMSG,7FEDB19D-70FE-A73...
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
3120B2A9-057E-E4E...,null,null,null,{5DD8F7FB-96F3-E6...,null,null,null,null,"{{null, null, nul...",{1908210266},{78},{5BA646C2-52E0-FD...,{21792},1523462095074000000,EVENT_SENDMSG,44F8544B-E70A-F80...
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
3120B2A9-057E-E4E...,null,null,null,{626CAE17-2D70-EF...,null,null,null,null,"{{null, null, nul...",{1908210269},{78},{6881BEAE-AEDC-B8...,{21790},1523462095074000000,EVENT_RECVMSG,534302DD-285B-93C...
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
3120B2A9-057E-E4E...,null,null,null,{C230CDA8-DACD-5B...,null,null,null,null,"{{null, null, nul...",{1908210279},{91},{6881BEAE-AEDC-B8...,{21790},1523462095074000000,EVENT_SENDMSG,D5305B97-7EA6-B18...
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [60]:
start_time = 1523318400 * 1000000000
end_time = 1523404800 * 1000000000
events_10th = events_10th.filter("timestampNanos > {} AND timestampNanos < {}".format(start_time, end_time))

In [61]:
events_10th

hostId,location,name,parameters,predicateObject,predicateObject2,predicateObject2Path,predicateObjectPath,programPoint,properties,sequence,size,subject,threadId,timestampNanos,type,uuid
3120B2A9-057E-E4E...,null,null,null,{5B0144A8-2B4B-0D...,null,null,null,null,"{{null, null, nul...",{1607898017},null,{470F9BDD-A889-BB...,{14478},1523362794679000000,EVENT_UNIT,58454F7E-B685-B14...
3120B2A9-057E-E4E...,null,null,null,{479B9A88-E866-88...,null,null,null,null,"{{null, null, nul...",{1508724850},{16384},{BA7EE6F5-7616-C1...,{18395},1523318400006000000,EVENT_WRITE,EBA27987-D238-D4F...
3120B2A9-057E-E4E...,null,null,null,{479B9A88-E866-88...,null,null,null,null,"{{null, null, nul...",{1508724851},{16384},{BA7EE6F5-7616-C1...,{18395},1523318400006000000,EVENT_WRITE,9A0AA88C-840E-A2F...
3120B2A9-057E-E4E...,null,null,null,{479B9A88-E866-88...,null,null,null,null,"{{null, null, nul...",{1508724852},{16372},{BA7EE6F5-7616-C1...,{18395},1523318400006000000,EVENT_WRITE,9901D5E0-BC08-B1B...
3120B2A9-057E-E4E...,null,null,null,{479B9A88-E866-88...,null,null,null,null,"{{null, null, nul...",{1508724853},{1928},{BA7EE6F5-7616-C1...,{18395},1523318400006000000,EVENT_WRITE,1A52402C-490B-023...
3120B2A9-057E-E4E...,null,null,null,{86068F31-5A39-AF...,null,null,null,null,"{{null, null, nul...",{1508724854},{64},{BA7EE6F5-7616-C1...,{18395},1523318400006000000,EVENT_RECVMSG,BA335D23-9FC7-E89...
3120B2A9-057E-E4E...,null,null,null,{86068F31-5A39-AF...,null,null,null,null,"{{null, null, nul...",{1508724855},{16384},{BA7EE6F5-7616-C1...,{18395},1523318400014000000,EVENT_WRITE,CFFD3B81-C829-FFF...
3120B2A9-057E-E4E...,null,null,null,{86068F31-5A39-AF...,null,null,null,null,"{{null, null, nul...",{1508724856},{16384},{BA7EE6F5-7616-C1...,{18395},1523318400014000000,EVENT_WRITE,CBC8B3D7-C0D7-FEA...
3120B2A9-057E-E4E...,null,null,null,{86068F31-5A39-AF...,null,null,null,null,"{{null, null, nul...",{1508724857},{16372},{BA7EE6F5-7616-C1...,{18395},1523318400014000000,EVENT_WRITE,E023BFC2-691D-2D0...
3120B2A9-057E-E4E...,null,null,null,{86068F31-5A39-AF...,null,null,null,null,"{{null, null, nul...",{1508724858},{1928},{BA7EE6F5-7616-C1...,{18395},1523318400014000000,EVENT_WRITE,ADC2074E-2D5C-79E...


In [66]:
files_all = df.select("datum.*").select("`com.bbn.tc.schema.avro.cdm18.FileObject`").withColumnRenamed("com.bbn.tc.schema.avro.cdm18.FileObject", "file").select("file.*")
files_all = files_all.filter('uuid IS NOT NULL')
files_all

baseObject,fileDescriptor,hashes,localPrincipal,peInfo,size,type,uuid
"{{446}, 3120B2A9-...",null,null,null,null,null,FILE_OBJECT_FILE,6C98E2EA-5756-658...
"{{447}, 3120B2A9-...",null,null,null,null,null,FILE_OBJECT_FILE,482A4610-A13D-443...
"{{3599}, 3120B2A9...",null,null,null,null,null,FILE_OBJECT_FILE,268BF4D5-321D-1E4...
"{{3596}, 3120B2A9...",null,null,null,null,null,FILE_OBJECT_FILE,92BFE0D5-507F-C8C...
"{{453}, 3120B2A9-...",null,null,null,null,null,FILE_OBJECT_FILE,81414DD2-EBFE-360...
"{{452}, 3120B2A9-...",null,null,null,null,null,FILE_OBJECT_FILE,7E903CC5-ECC2-9AB...
"{{0}, 3120B2A9-05...",null,null,null,null,null,FILE_OBJECT_FILE,93005F23-3BD2-05D...
"{{0}, 3120B2A9-05...",null,null,null,null,null,FILE_OBJECT_FILE,B2C30F13-B6BF-978...
"{{0}, 3120B2A9-05...",null,null,null,null,null,FILE_OBJECT_FILE,865752A5-DD4B-178...
"{{0}, 3120B2A9-05...",null,null,null,null,null,FILE_OBJECT_FILE,FFA093C7-D5C5-5AF...


In [65]:
sockets_all = df.select("datum.*").select("`com.bbn.tc.schema.avro.cdm18.NetFlowObject`").withColumnRenamed("com.bbn.tc.schema.avro.cdm18.NetFlowObject", "socket").select("socket.*")
sockets_all = sockets_all.filter('uuid IS NOT NULL')
sockets_all

baseObject,fileDescriptor,ipProtocol,localAddress,localPort,remoteAddress,remotePort,uuid
"{{571341}, 3120B2...",null,{17},0.0.0.0,5353,128.55.12.172,5353,233888CD-FBED-ABD...
"{{882462}, 3120B2...",null,{17},0.0.0.0,5353,128.55.12.170,5353,8D970067-EB67-DD1...
"{{890796}, 3120B2...",null,{17},0.0.0.0,5353,128.55.12.171,5353,775624C1-2807-BFF...
"{{571342}, 3120B2...",null,{17},0.0.0.0,5353,128.55.12.172,5353,AFD5C7CD-5DBD-D2D...
"{{882463}, 3120B2...",null,{17},0.0.0.0,5353,128.55.12.170,5353,826FA86C-E4A9-E41...
"{{890797}, 3120B2...",null,{17},0.0.0.0,5353,128.55.12.171,5353,12E2ED15-4293-55F...
"{{571343}, 3120B2...",null,{17},0.0.0.0,5353,128.55.12.172,5353,D9E241AC-BF17-E94...
"{{23}, 3120B2A9-0...",null,{17},128.55.12.118,58228,128.55.12.10,53,DD8D8BA6-8E9E-D45...
"{{24}, 3120B2A9-0...",null,{17},128.55.12.118,58228,128.55.12.10,53,5DB4F427-EC10-733...
"{{571344}, 3120B2...",null,{17},0.0.0.0,5353,128.55.12.172,5353,91E88749-AA86-25B...


In [67]:
unique_users = subjects_10th.select('localPrincipal').distinct()
unique_users.count()

20

In [68]:
files_all.count()

1159603

In [69]:
sockets_all.count()

4206479

In [70]:
trace_red_labels = []
with open('../groundtruth_threaTrace/trace.txt', 'r') as f:
    trace_red_labels = f.read().splitlines()

In [89]:
red_uuids = str("('" + "', '".join(trace_red_labels) + "')")

In [90]:
events_10th.printSchema()

root
 |-- hostId: string (nullable = true)
 |-- location: struct (nullable = true)
 |    |-- long: long (nullable = true)
 |-- name: string (nullable = true)
 |-- parameters: string (nullable = true)
 |-- predicateObject: struct (nullable = true)
 |    |-- com.bbn.tc.schema.avro.cdm18.UUID: string (nullable = true)
 |-- predicateObject2: struct (nullable = true)
 |    |-- com.bbn.tc.schema.avro.cdm18.UUID: string (nullable = true)
 |-- predicateObject2Path: struct (nullable = true)
 |    |-- string: string (nullable = true)
 |-- predicateObjectPath: struct (nullable = true)
 |    |-- string: string (nullable = true)
 |-- programPoint: string (nullable = true)
 |-- properties: struct (nullable = true)
 |    |-- map: struct (nullable = true)
 |    |    |-- flags: string (nullable = true)
 |    |    |-- mode: string (nullable = true)
 |    |    |-- operation: string (nullable = true)
 |    |    |-- opm: string (nullable = true)
 |    |    |-- protection: string (nullable = true)
 |-- sequ

In [91]:
red_events_10th = events_10th.filter('uuid IN ' + red_uuids)
red_events_10th

23/02/01 11:12:57 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


23/02/01 11:13:00 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


23/02/01 11:13:03 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


23/02/01 11:13:07 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


23/02/01 11:13:28 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


...

23/02/01 11:29:44 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


23/02/01 11:30:06 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


23/02/01 11:31:47 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


23/02/01 11:39:55 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


hostId,location,name,parameters,predicateObject,predicateObject2,predicateObject2Path,predicateObjectPath,programPoint,properties,sequence,size,subject,threadId,timestampNanos,type,uuid


In [92]:
unique_hosts = subjects_10th.select('hostId').distinct()
unique_hosts.count()

1

In [93]:
red_events_10th.count()

23/02/01 12:01:16 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


0

In [94]:
subjects_10th.printSchema()

root
 |-- cid: long (nullable = true)
 |-- cmdLine: struct (nullable = true)
 |    |-- string: string (nullable = true)
 |-- count: struct (nullable = true)
 |    |-- int: long (nullable = true)
 |-- exportedLibraries: string (nullable = true)
 |-- hostId: string (nullable = true)
 |-- importedLibraries: string (nullable = true)
 |-- iteration: struct (nullable = true)
 |    |-- int: long (nullable = true)
 |-- localPrincipal: string (nullable = true)
 |-- parentSubject: struct (nullable = true)
 |    |-- com.bbn.tc.schema.avro.cdm18.UUID: string (nullable = true)
 |-- privilegeLevel: string (nullable = true)
 |-- properties: struct (nullable = true)
 |    |-- map: struct (nullable = true)
 |    |    |-- cwd: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- ppid: string (nullable = true)
 |    |    |-- seen time: string (nullable = true)
 |-- startTimestampNanos: long (nullable = true)
 |-- type: string (nullable = true)
 |-- unitId: struct (nullabl

In [95]:
files_all.printSchema()

root
 |-- baseObject: struct (nullable = true)
 |    |-- epoch: struct (nullable = true)
 |    |    |-- int: long (nullable = true)
 |    |-- hostId: string (nullable = true)
 |    |-- permission: struct (nullable = true)
 |    |    |-- com.bbn.tc.schema.avro.cdm18.SHORT: string (nullable = true)
 |    |-- properties: struct (nullable = true)
 |    |    |-- map: struct (nullable = true)
 |    |    |    |-- path: string (nullable = true)
 |-- fileDescriptor: string (nullable = true)
 |-- hashes: string (nullable = true)
 |-- localPrincipal: string (nullable = true)
 |-- peInfo: string (nullable = true)
 |-- size: string (nullable = true)
 |-- type: string (nullable = true)
 |-- uuid: string (nullable = true)



In [96]:
sockets_all.printSchema()

root
 |-- baseObject: struct (nullable = true)
 |    |-- epoch: struct (nullable = true)
 |    |    |-- int: long (nullable = true)
 |    |-- hostId: string (nullable = true)
 |    |-- permission: string (nullable = true)
 |-- fileDescriptor: string (nullable = true)
 |-- ipProtocol: struct (nullable = true)
 |    |-- int: long (nullable = true)
 |-- localAddress: string (nullable = true)
 |-- localPort: long (nullable = true)
 |-- remoteAddress: string (nullable = true)
 |-- remotePort: long (nullable = true)
 |-- uuid: string (nullable = true)



In [97]:
red_files = files_all.filter('uuid IN ' + red_uuids)
red_files

23/02/01 12:48:02 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


23/02/01 12:48:05 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


23/02/01 12:48:09 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


23/02/01 12:48:14 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


23/02/01 12:48:35 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


...

23/02/01 13:14:01 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


23/02/01 13:16:03 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


23/02/01 13:18:32 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


23/02/01 13:21:40 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


baseObject,fileDescriptor,hashes,localPrincipal,peInfo,size,type,uuid
"{{0}, E621F964-5A...",null,null,null,null,null,FILE_OBJECT_FILE,49463062-60DC-4F2...
"{{0}, E621F964-5A...",null,null,null,null,null,FILE_OBJECT_FILE,3FAC9ED8-FB1F-F59...
"{{0}, E621F964-5A...",null,null,null,null,null,FILE_OBJECT_FILE,ADCB354C-BAA9-51E...
"{{0}, E621F964-5A...",null,null,null,null,null,FILE_OBJECT_FILE,7AC6F45A-FA2D-0FF...
"{{0}, E621F964-5A...",null,null,null,null,null,FILE_OBJECT_DIR,6FC69B3D-1E90-223...
"{{0}, E621F964-5A...",null,null,null,null,null,FILE_OBJECT_FILE,93E0C854-95A4-FE1...
"{{0}, E621F964-5A...",null,null,null,null,null,FILE_OBJECT_FILE,96FE4223-D38F-9D4...
"{{0}, E621F964-5A...",null,null,null,null,null,FILE_OBJECT_FILE,3C8B4DE3-9032-391...
"{{1}, E621F964-5A...",null,null,null,null,null,FILE_OBJECT_FILE,50FB8D0F-2D9E-7C6...
"{{1}, E621F964-5A...",null,null,null,null,null,FILE_OBJECT_FILE,BBC43AE7-8DF9-49D...


In [98]:
red_files.count()

23/02/01 13:23:05 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


19

In [100]:
red_subjects_10th = subjects_10th.filter('uuid IN ' + red_uuids)
red_subjects_10th

23/02/01 14:11:12 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


23/02/01 14:11:15 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


23/02/01 14:11:19 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


23/02/01 14:11:24 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


23/02/01 14:11:46 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


...

23/02/01 14:29:06 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


23/02/01 14:29:29 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


23/02/01 14:31:15 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


23/02/01 14:40:00 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


cid,cmdLine,count,exportedLibraries,hostId,importedLibraries,iteration,localPrincipal,parentSubject,privilegeLevel,properties,startTimestampNanos,type,unitId,uuid


In [101]:
red_subjects_10th.count()

23/02/01 14:46:51 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


0

In [102]:
red_sockets = sockets_all.filter('uuid IN ' + red_uuids)
red_sockets

23/02/01 15:04:56 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


23/02/01 15:04:59 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


23/02/01 15:05:02 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


23/02/01 15:05:09 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


23/02/01 15:05:34 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


...

23/02/01 15:07:34 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


23/02/01 15:07:37 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


23/02/01 15:07:43 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


23/02/01 15:08:09 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


baseObject,fileDescriptor,ipProtocol,localAddress,localPort,remoteAddress,remotePort,uuid
"{{0}, E621F964-5A...",null,{6},128.55.12.118,59065,162.66.239.75,80,7E6F9A12-EDFA-C87...
"{{0}, E621F964-5A...",null,{6},128.55.12.118,49701,128.55.12.73,22,87EBF3B7-6691-15A...
"{{0}, E621F964-5A...",null,{6},128.55.12.118,42911,128.55.12.73,23,DC8E2FA9-7C85-11F...
"{{0}, E621F964-5A...",null,{6},128.55.12.118,50608,128.55.12.73,24,E30A436D-A0AE-660...
"{{0}, E621F964-5A...",null,{6},128.55.12.118,55040,128.55.12.73,25,94BFCBB6-5C7C-857...
"{{0}, E621F964-5A...",null,{6},128.55.12.118,48204,128.55.12.73,26,42A3BFC6-7F80-019...
"{{0}, E621F964-5A...",null,{6},128.55.12.118,50480,128.55.12.73,27,E76BD3D0-F400-BE7...
"{{0}, E621F964-5A...",null,{6},128.55.12.118,59202,128.55.12.73,28,F5E07F96-EDF3-7D7...
"{{0}, E621F964-5A...",null,{6},128.55.12.118,42089,128.55.12.73,29,BD034B68-4B7C-CD7...
"{{0}, E621F964-5A...",null,{6},128.55.12.118,41230,128.55.12.73,30,A6DB3A81-3D40-4F0...


In [103]:
red_sockets.count()

23/02/01 15:10:06 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


68061

In [111]:
unique_hosts = sockets_all.select('baseObject.hostId').distinct()
unique_hosts.count()

2

In [112]:
len(trace_red_labels)

68265

In [119]:
host_to_ip = sockets_all.select('baseObject.hostId', 'localAddress').distinct()
host_to_ip

hostId,localAddress
3120B2A9-057E-E4E...,0.0.0.0
3120B2A9-057E-E4E...,128.55.12.118
3120B2A9-057E-E4E...,0000:0000:0000:00...
3120B2A9-057E-E4E...,0000:0000:0000:00...
3120B2A9-057E-E4E...,0000:0000:0000:00...
3120B2A9-057E-E4E...,10.0.6.68
3120B2A9-057E-E4E...,127.0.0.1
E621F964-5A66-0F8...,0000:0000:0000:00...
E621F964-5A66-0F8...,0000:0000:0000:00...
E621F964-5A66-0F8...,10.0.6.68


In [121]:
unique_event_types = events_10th.select('type').distinct()
unique_event_types

type
EVENT_UNIT
EVENT_LINK
EVENT_MMAP
EVENT_WRITE
EVENT_CLOSE
EVENT_OPEN
EVENT_EXIT
EVENT_READ
EVENT_ACCEPT
EVENT_SENDMSG


In [122]:
unique_event_types.count()

25

In [123]:
red_files.select('baseObject.properties.map.path')

23/02/02 17:44:08 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


23/02/02 17:44:11 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


23/02/02 17:44:15 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


23/02/02 17:44:40 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


23/02/02 17:48:25 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


...

23/02/02 20:04:55 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


23/02/02 20:12:17 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


23/02/02 20:22:08 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


23/02/02 20:35:35 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


IOPub message rate exceeded.==>                                (183 + 20) / 478]
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [125]:
unique_event_types.show(50)

+--------------------+
|                type|
+--------------------+
|          EVENT_UNIT|
|          EVENT_LINK|
|          EVENT_MMAP|
|         EVENT_WRITE|
|         EVENT_CLOSE|
|          EVENT_OPEN|
|          EVENT_EXIT|
|          EVENT_READ|
|        EVENT_ACCEPT|
|       EVENT_SENDMSG|
|          EVENT_FORK|
|        EVENT_UNLINK|
|       EVENT_EXECUTE|
|       EVENT_RECVMSG|
|EVENT_MODIFY_FILE...|
| EVENT_CREATE_OBJECT|
|        EVENT_RENAME|
|       EVENT_CONNECT|
|   EVENT_LOADLIBRARY|
|      EVENT_TRUNCATE|
|         EVENT_CLONE|
|      EVENT_MPROTECT|
|         EVENT_OTHER|
|EVENT_CHANGE_PRIN...|
|        EVENT_UPDATE|
+--------------------+

